<a href="https://colab.research.google.com/github/frios2020/ETL-MADE-EASY/blob/main/Registro_de_Actuaciones_2016_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ETL Processed paperworks at the Peruvian Consulate in Paterson New Jersey during 2016 - 2022.





In [ ]:
import pandas as pd              # this module helps in processing CSV files
import glob                      # this module helps in selecting files 
from datetime import datetime    # this module helps to manipulate datetime fields 
from google.colab import drive   # this module helps to connect to google drive

## Set Paths


In [ ]:
logfile    = "logfile.txt"                                        # all event logs will be stored in this file
targetfile = "transformed_data.csv"                               # file where transformed data is stored
folder     = "/content/drive/MyDrive/RAW DATA/ACTUACIONES/*.xlsx" # path in google drive where the files to process are.

## Connecting Google Drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
files = glob.glob(folder)
print("Total number of files: ", len(files))

Total number of files:  7


In [ ]:
for x in files:
  print(x)

/content/drive/MyDrive/RAW DATA/ACTUACIONES/RegistroGeneral_2016.xlsx
/content/drive/MyDrive/RAW DATA/ACTUACIONES/RegistroGeneral_2017.xlsx
/content/drive/MyDrive/RAW DATA/ACTUACIONES/RegistroGeneral_2018.xlsx
/content/drive/MyDrive/RAW DATA/ACTUACIONES/RegistroGeneral_2019.xlsx
/content/drive/MyDrive/RAW DATA/ACTUACIONES/RegistroGeneral_2020.xlsx
/content/drive/MyDrive/RAW DATA/ACTUACIONES/RegistroGeneral_2021.xlsx
/content/drive/MyDrive/RAW DATA/ACTUACIONES/RegistroGeneral_2022.xlsx


# Extract

  # XLS Extract Function

In [ ]:
def extract_from_xls(file_to_process):
    dataframe = pd.read_excel(file_to_process, parse_dates=['Fecha Actuación']) # Parametrer parse_dates make "Fecha Actuacion" like datetime.
    return dataframe

In [ ]:
def extract():
    extracted_data = pd.DataFrame()                           # Create an empty data frame to hold extracted data
    for xlsfile in glob.glob(folder):                         # Process all xls files and add in onedataframe using append function.
        extracted_data = extracted_data.append(extract_from_xls(xlsfile), ignore_index=True)
    return extracted_data

## Cleaning and transforming data

In [ ]:
#from numpy import datetime64
def transform(data):
  data = data.dropna(axis=1,how='all')                                       # Delete columns where all values are NaN.
  data['Fecha'] = data['Fecha Actuación'].dt.date                            # Create a field "fecha"
  data['Hora'] = pd.to_datetime(data['Fecha Actuación']).dt.strftime('%H:%M')# Create a field "hora".
  data['Anio']=data['Fecha Actuación'].dt.year                               # Create a field "anio".
  data['Mes']=data['Fecha Actuación'].dt.month                               # Create a field "mes".
  data['Dia']=data['Fecha Actuación'].dt.dayofweek                           # Create a fiekd "dia" Monday is 0 and Sunday is 6
  
  # Rename columns
  data.rename(columns = {'N° Item':'Num_Item', 'Corr. Actuación':'Corr_General',
                                 'Fecha Actuación':'Fecha_Actuacion','Nombre del Interesado':'Nombres',
                                 'Autoadhesivo Consular':'Autoadhesivo_Consular','Naturaleza del Acto':'Descripcion_Actuacion',
                                 'N° Tarifa':'Num_Tarifa','N° Actuación':'Num_Actuacion','Moneda Extranjera $':'Moneda_Extranjera',
                                 'Soles Consular S/C':'Soles_Consulares','T. C. Consular':'TC_Consular','Observación':'Observacion'
                                 }, inplace = True)
  # Fix data types of columns
  data['Fecha_Actuacion'] = data['Fecha_Actuacion'].astype('datetime64[m]') # Just hours and minutes.
  data['Num_Item']=data.Num_Item.astype(int)
  data['Corr_General']=data.Corr_General.astype(int)
  data['Autoadhesivo_Consular']=data.Autoadhesivo_Consular.astype(str)
  data['Num_Actuacion']=data.Num_Actuacion.astype(int)
  data['Moneda_Extranjera']=data.Moneda_Extranjera.astype(int)
  data['Soles_Consulares']=data.Soles_Consulares.astype(int)
  data['TC_Consular']=data.TC_Consular.astype(int)

  return data

## Load data

In [ ]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile, encoding="utf-16")

## Logging

In [ ]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL Process


In [ ]:
log("ETL Job Started")

In [ ]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,N° Item,Corr. Actuación,Unnamed: 2,Fecha Actuación,Nombre del Interesado,Unnamed: 5,Autoadhesivo Consular,Unnamed: 7,Naturaleza del Acto,Unnamed: 9,N° Tarifa,Unnamed: 11,Unnamed: 12,N° Actuación,Moneda Extranjera $,Unnamed: 15,Unnamed: 16,Soles Consular S/C,T. C. Consular,Observación
0,1.0,1.0,NaN,2016-03-01 09:58:04.617,CHAVEZ ALBUJAR JUAN MARTIN,NaN,42301.0,NaN,POR EXPEDICION DUPLICADO DNI,NaN,71A,NaN,NaN,102.0,6.0,NaN,NaN,6.0,1.0,EFECTIVO
1,2.0,3.0,NaN,2016-03-01 10:07:18.163,HILARES HUAMANI MARIO JOSE,NaN,42701.0,NaN,RENOVACION PASAPORTE 5 AÑOS,NaN,61B,NaN,NaN,549.0,35.0,NaN,NaN,35.0,1.0,EFECTIVO
2,3.0,4.0,NaN,2016-03-01 10:09:35.267,LOBATON GUTIERREZ JESUS SERGIO ALEJANDRO,NaN,42801.0,NaN,CERTIFICADO DE SUPERVIVENCIA,NaN,28A,NaN,NaN,487.0,1.0,NaN,NaN,1.0,1.0,EFECTIVO
3,4.0,5.0,NaN,2016-03-01 10:20:41.810,ORTIZ DE AZABACHE IRMA AMELIA,NaN,42302.0,NaN,POR RENOVACION DNI CADUCADOS,NaN,69B,NaN,NaN,428.0,7.0,NaN,NaN,7.0,1.0,EFECTIVO
4,5.0,7.0,NaN,2016-03-01 10:30:58.340,AZABACHE ESPEJO EDUARDO BERNARDO,NaN,42702.0,NaN,RENOVACION PASAPORTE 5 AÑOS,NaN,61B,NaN,NaN,550.0,35.0,NaN,NaN,35.0,1.0,EFECTIVO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195365,32842.0,33229.0,NaN,2022-12-30 13:48:29.847,LLONTOP LOPEZ EDUARDO ANTONIO,NaN,3606058.0,NaN,POR EXPEDICIÓN DE UN PASAPORTE ELECTRÓNICO ORD...,NaN,59B,NaN,NaN,9929.0,70.0,NaN,NaN,70.0,1.0,EFECTIVO
195366,32843.0,33230.0,NaN,2022-12-30 13:48:58.673,LLONTOP LOPEZ EDUARDO ANTONIO,NaN,3606396.0,NaN,POR LA RECTIFICACION DE DNI,NaN,72A1,NaN,NaN,2244.0,6.0,NaN,NaN,6.0,1.0,EFECTIVO
195367,32844.0,33231.0,NaN,2022-12-30 13:54:39.070,FELIPE PONCE CARLA VERONICA,NaN,3601977.0,NaN,PODER GENERAL,NaN,12A,NaN,NaN,373.0,45.0,NaN,NaN,45.0,1.0,EFECTIVO
195368,32845.0,33232.0,NaN,2022-12-30 13:55:08.257,FELIPE PONCE CARLA VERONICA,NaN,3601978.0,NaN,TESTIMONIO DE ESCRITURA PUBLICA,NaN,17A,NaN,NaN,1130.0,48.0,NaN,NaN,48.0,1.0,EFECTIVO


In [ ]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

<ipython-input-8-3b1f0db3faaa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Fecha'] = data['Fecha Actuación'].dt.date                            # Creamos un campo con solo fecha
<ipython-input-8-3b1f0db3faaa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Hora'] = pd.to_datetime(data['Fecha Actuación']).dt.strftime('%H:%M')# Creamos un campo con solo hora.
<ipython-input-8-3b1f0db3faaa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,Num_Item,Corr_General,Fecha_Actuacion,Nombres,Autoadhesivo_Consular,Descripcion_Actuacion,Num_Tarifa,Num_Actuacion,Moneda_Extranjera,Soles_Consulares,TC_Consular,Observacion,Fecha,Hora,Anio,Mes,Dia
0,1,1,2016-03-01 09:58:00,CHAVEZ ALBUJAR JUAN MARTIN,42301.0,POR EXPEDICION DUPLICADO DNI,71A,102,6,6,1,EFECTIVO,2016-03-01,09:58,2016,3,1
1,2,3,2016-03-01 10:07:00,HILARES HUAMANI MARIO JOSE,42701.0,RENOVACION PASAPORTE 5 AÑOS,61B,549,35,35,1,EFECTIVO,2016-03-01,10:07,2016,3,1
2,3,4,2016-03-01 10:09:00,LOBATON GUTIERREZ JESUS SERGIO ALEJANDRO,42801.0,CERTIFICADO DE SUPERVIVENCIA,28A,487,1,1,1,EFECTIVO,2016-03-01,10:09,2016,3,1
3,4,5,2016-03-01 10:20:00,ORTIZ DE AZABACHE IRMA AMELIA,42302.0,POR RENOVACION DNI CADUCADOS,69B,428,7,7,1,EFECTIVO,2016-03-01,10:20,2016,3,1
4,5,7,2016-03-01 10:30:00,AZABACHE ESPEJO EDUARDO BERNARDO,42702.0,RENOVACION PASAPORTE 5 AÑOS,61B,550,35,35,1,EFECTIVO,2016-03-01,10:30,2016,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195365,32842,33229,2022-12-30 13:48:00,LLONTOP LOPEZ EDUARDO ANTONIO,3606058.0,POR EXPEDICIÓN DE UN PASAPORTE ELECTRÓNICO ORD...,59B,9929,70,70,1,EFECTIVO,2022-12-30,13:48,2022,12,4
195366,32843,33230,2022-12-30 13:48:00,LLONTOP LOPEZ EDUARDO ANTONIO,3606396.0,POR LA RECTIFICACION DE DNI,72A1,2244,6,6,1,EFECTIVO,2022-12-30,13:48,2022,12,4
195367,32844,33231,2022-12-30 13:54:00,FELIPE PONCE CARLA VERONICA,3601977.0,PODER GENERAL,12A,373,45,45,1,EFECTIVO,2022-12-30,13:54,2022,12,4
195368,32845,33232,2022-12-30 13:55:00,FELIPE PONCE CARLA VERONICA,3601978.0,TESTIMONIO DE ESCRITURA PUBLICA,17A,1130,48,48,1,EFECTIVO,2022-12-30,13:55,2022,12,4


In [ ]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [ ]:
log("ETL Job Ended")